# 1. Descargar los datos de label studio
* Recodar exportar las etiquetas en formato YOLO y guardarlas en su carpeta respectivas

In [1]:
import os
import requests
from dotenv import load_dotenv
from label_studio_sdk import Client

load_dotenv()  # Carga variables desde .env

LABEL_STUDIO_URL = os.getenv("LABEL_STUDIO_URL")
API_KEY = os.getenv("LABEL_STUDIO_API_KEY")

In [4]:
# Inicializar el cliente
ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)

# Obtener todos los proyectos
projects = ls.get_projects()

# Imprimir detalles de cada proyecto
print(f"Proyectos encontrados: {len(projects)}")
for project in projects:
    print(f"ID: {project.id} - Nombre: {project.get_params()['title']}")

Proyectos encontrados: 7
ID: 11 - Nombre: ParlVotingZones
ID: 10 - Nombre: TesisFilas
ID: 6 - Nombre: TesisAsistencia
ID: 5 - Nombre: TesisVotacion
ID: 4 - Nombre: TCG-Straight
ID: 3 - Nombre: TCG-cuadros
ID: 2 - Nombre: TCG-Segmentation


In [5]:
def download_images_from_label_studio(project_id: int, output_dir: str):
    # Crear carpeta de salida si no existe
    os.makedirs(output_dir, exist_ok=True)

    # Conectar con Label Studio
    ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
    project = ls.get_project(project_id)
    tasks = project.get_tasks()
    total = len(tasks)
    print(f"Total imágenes encontradas: {total}")
    print(f"Descargando {total} imágenes...")

    for i, task in enumerate(tasks, start=1):
        image_path = task['data'].get('image')
        if image_path:
            full_url = f"{LABEL_STUDIO_URL}{image_path}"
            file_name = os.path.basename(image_path)
            output_path = os.path.join(output_dir, file_name)

            response = requests.get(full_url, headers={'Authorization': f'Token {API_KEY}'}, stream=True)
            if response.status_code == 200:
                with open(output_path, 'wb') as f:
                    for chunk in response.iter_content(chunk_size=8192):
                        f.write(chunk)
                print(f"[{i}/{total}] ✅ Descargado: {file_name}")
            else:
                print(f"[{i}/{total}] ❌ Error al descargar {file_name}: {response.status_code}")
        else:
            print(f"[{i}/{total}] ⚠️ Tarea sin imagen asociada")

In [6]:
# Votación
download_images_from_label_studio(project_id=11, output_dir='../data/dataset_etiquetado_zonas/d_etiquetados/images')

Total imágenes encontradas: 300
Descargando 300 imágenes...
[1/300] ✅ Descargado: 860a45e8-187_pp2021_2026_pa2022_2023_leg1_page_5.jpg
[2/300] ✅ Descargado: e0842ac0-188_pp2021_2026_pa2022_2023_leg1_page_30.jpg
[3/300] ✅ Descargado: 20ce971f-188_pp2021_2026_pa2022_2023_leg1_page_55.jpg
[4/300] ✅ Descargado: dddf1316-192_pp2021_2026_pa2022_2023_leg1_page_3.jpg
[5/300] ✅ Descargado: 47b056ff-206_pp2021_2026_pa2021_2022_leg2_page_60.jpg
[6/300] ✅ Descargado: 46621042-207_pp2021_2026_pa2021_2022_leg2_page_5.jpg
[7/300] ✅ Descargado: 5dad3509-207_pp2021_2026_pa2021_2022_leg2_page_32.jpg
[8/300] ✅ Descargado: 121a5187-207_pp2021_2026_pa2021_2022_leg2_page_34.jpg
[9/300] ✅ Descargado: 9451d2be-208_pp2021_2026_pa2021_2022_leg2_page_18.jpg
[10/300] ✅ Descargado: 46445d41-210_pp2021_2026_pa2021_2022_leg2_page_15.jpg
[11/300] ✅ Descargado: 3cebd8be-212_pp2021_2026_pa2021_2022_leg2_page_29.jpg
[12/300] ✅ Descargado: cd54c542-213_pp2021_2026_pa2021_2022_leg2_page_6.jpg
[13/300] ✅ Descargado: 2cb36a

# B. Probar máscaras


In [ ]:
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def visualizar_yolo_labels(nombre_base: str,
                           images_dir='../data/dataset_etiquetado_zonas/d_etiquetados/images',
                           labels_dir='../data/dataset_etiquetado_zonas/d_etiquetados/labels',
                           classes_file='../data/dataset_etiquetado_zonas/d_etiquetados/classes.txt',
                           ext='.jpg'):
    """
    Visualiza una imagen junto con sus anotaciones YOLO.

    Args:
        nombre_base (str): Nombre del archivo sin extensión (por ejemplo: '0ace00af-172_votacion').
        images_dir (str): Ruta al directorio de imágenes.
        labels_dir (str): Ruta al directorio de etiquetas.
        classes_file (str): Ruta al archivo con nombres de clases.
        ext (str): Extensión de la imagen (por defecto '.jpg').
    """
    # Construir rutas
    image_path = os.path.join(images_dir, nombre_base + ext)
    label_path = os.path.join(labels_dir, nombre_base + '.txt')

    # Leer clases
    with open(classes_file, 'r', encoding='utf-8') as f:
        classes = [line.strip() for line in f.readlines()]

    # Leer imagen
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {image_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape

    # Crear figura
    fig, ax = plt.subplots(1, figsize=(10, 10))
    ax.imshow(image)

    # Leer etiquetas
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue  # línea inválida
                class_id = int(parts[0])
                cx, cy, bw, bh = map(float, parts[1:])

                # Convertir a coordenadas absolutas
                x = (cx - bw / 2) * w
                y = (cy - bh / 2) * h
                bw = bw * w
                bh = bh * h

                # Dibujar rectángulo
                rect = patches.Rectangle((x, y), bw, bh, linewidth=2,
                                         edgecolor='lime', facecolor='none')
                ax.add_patch(rect)

                # Añadir nombre de clase
                label = classes[class_id] if class_id < len(classes) else f"Clase {class_id}"
                ax.text(x, y - 5, label, color='lime', fontsize=12,
                        bbox=dict(facecolor='black', alpha=0.5, edgecolor='none'))
    else:
        print(f"⚠️ No se encontró la etiqueta: {label_path}")

    ax.axis('off')
    ax.set_title(f"Visualización: {nombre_base + ext}")
    plt.show()

In [9]:
visualizar_yolo_labels('0b0eecf1-288_pp2016_2021_pa2021_2022_leg4_page_5')


FileNotFoundError: [Errno 2] No such file or directory: '../data/d_etiquetados/classes.txt'